<a href="https://colab.research.google.com/github/lorquez/real_estate_price_predictor/blob/main/Data_Cleansing_real_estate_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are the modules we will need for this project

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import json
import math
import os

Let's import the database and check it out

In [4]:
datasets_folder = '/content/drive/MyDrive/Kschool_TFM/datasets/'

In [5]:
idealista_df = pd.read_csv(f'{datasets_folder}idealista_dataset.csv')
idealista_df.sample(10)

,id,propertyType,title,locationId,price,size,hasParking,roomNumber,bathNumber,hasSwimmingPool,...,hasGarden,hasLift,isGoodCondition,isNeedsRenovating,isNewDevelopment,featureTags,heatingType,interiorExterior,floor,area_name
8514,95252195,chalet,chalet adosado en venta en carretera rafael villa,0-EU-ES-28-07-001-079-09,990000,430.0,1,4,5,1,...,1,0,1,0,0,['Lujo'],Calefacción individual: Gasoil,does not apply,does not apply,Moncloa
9668,98093241,piso,piso en venta en calle alto del esparragal & j...,0-EU-ES-28-07-001-079-19,359900,119.0,1,3,2,0,...,0,1,0,0,1,[],no info,exterior,planta 5ª,Vicálvaro
10107,91159199,piso,"dúplex en venta en calle del barco, 45",0-EU-ES-28-07-001-079-01,250000,60.0,0,1,1,0,...,0,1,1,0,0,[],no info,exterior,bajo,Centro
8966,97788973,piso,"piso en venta en calle collado de cerro malejo, 6",0-EU-ES-28-07-001-079-08,560000,115.0,0,3,2,1,...,1,1,1,0,0,[],Calefacción individual: Gas natural,interior,planta 1ª,Fuencarral
11387,97461211,piso,ático en venta en goya,0-EU-ES-28-07-001-079-04,1750000,200.0,0,4,3,0,...,0,1,1,0,0,['Lujo'],Calefacción central: Gas,exterior,planta 6ª,Barrio de Salamanca
11212,97997614,piso,piso en venta en calle de santa virgilia,0-EU-ES-28-07-001-079-16,380000,115.0,0,3,2,0,...,0,1,1,0,0,[],Calefacción central: Gas,exterior,planta 1ª,Hortaleza
5246,97491583,piso,piso en venta en lista,0-EU-ES-28-07-001-079-04,860000,154.0,0,4,3,0,...,0,1,0,1,0,[],Calefacción central,exterior,planta 2ª,Barrio de Salamanca
12312,97694152,piso,piso en venta en calle mariano fernández,0-EU-ES-28-07-001-079-06,220000,60.0,0,2,1,0,...,0,0,1,0,0,['Apartamento'],Calefacción individual: Eléctrica,interior,planta 3ª,Tetuán
7106,98362207,piso,piso en venta en calle de preciados,0-EU-ES-28-07-001-079-01,955000,168.0,0,5,1,0,...,0,1,0,1,0,[],no info,exterior,planta 2ª,Centro
1879,96810291,piso,piso en venta en calle de carlos martín álvarez,0-EU-ES-28-07-001-079-13,185000,100.0,0,4,1,0,...,0,1,1,0,0,[],Calefacción individual: Gas natural,exterior,planta 1ª,Puente de Vallecas


In [6]:
idealista_df.isna().any()

id                   False
propertyType         False
title                False
locationId           False
price                False
size                 False
hasParking           False
roomNumber           False
bathNumber           False
hasSwimmingPool      False
hasTerrace           False
hasGarden            False
hasLift              False
isGoodCondition      False
isNeedsRenovating    False
isNewDevelopment     False
featureTags          False
heatingType          False
interiorExterior     False
floor                False
area_name            False
dtype: bool

Looks like idealista dataset has no NaN value.

In [7]:
idealista_df.dtypes

id                     int64
propertyType          object
title                 object
locationId            object
price                  int64
size                 float64
hasParking             int64
roomNumber             int64
bathNumber             int64
hasSwimmingPool        int64
hasTerrace             int64
hasGarden              int64
hasLift                int64
isGoodCondition        int64
isNeedsRenovating      int64
isNewDevelopment       int64
featureTags           object
heatingType           object
interiorExterior      object
floor                 object
area_name             object
dtype: object

**Fotocasa Data Cleansing**

In [8]:
fotocasa_df = pd.read_csv(f'{datasets_folder}fotocasa_dataset.csv')
fotocasa_df.sample(10)

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
24782,fotocasa,426,Ático en venta en Calle de José Ortega y Gasset,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Barrio de Salamanca,Lista,Calle de José Ortega y Gasset,28006,...,91,G,Gas Natural,NaN,Muy bien,NaN,12,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",575000
11846,fotocasa,339,Piso en venta en Calle del Hierro,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Arganzuela,Legazpi,Calle del Hierro,28045,...,118,G,NaN,NaN,Bien,10 a 20 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",429999
17251,fotocasa,549,Piso en venta en Calle de Arturo Soria,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Ciudad Lineal,Costillares,Calle de Arturo Soria,28033,...,168,E,NaN,NaN,Bien,30 a 50 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",775000
2645,fotocasa,383,Piso en venta en Calle del Collado del Hornillo,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Fuencarral - El Pardo,Arroyo del Fresno,Calle del Collado del Hornillo,28035,...,142,D,NaN,NaN,Muy bien,20 a 30 años,0,0,"{'2': 'Armarios', '3': 'Calefacción', '9': 'Pa...",587000
28124,fotocasa,137,Casa o chalet en venta en Carretera de Colmena...,https://www.fotocasa.es/es/comprar/vivienda/ga...,https://static.inmofactory.com/images/inmofact...,España,La Canaleja,NaN,"Carretera de Colmenarejo, 45",28260,...,450,G,NaN,NaN,A reformar,30 a 50 años,5,0,"{'7': 'Jardín Privado', '17': 'Piscina'}",600000
21466,fotocasa,209,"Piso en venta en Calle de Palencia, 1",https://www.fotocasa.es/es/comprar/vivienda/to...,https://static.inmofactory.com/images/inmofact...,España,Centro,NaN,"Calle de Palencia, 1",28850,...,86,G,NaN,NaN,Muy bien,NaN,9,0,"{'2': 'Armarios', '3': 'Calefacción', '6': 'Gr...",119500
14449,fotocasa,706,Piso en venta en Paseo de la Reina Cristina,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Retiro,Pacífico,Paseo de la Reina Cristina,28014,...,147,G,NaN,NaN,Bien,NaN,9,0,"{'1': 'Aire acondicionado', '3': 'Calefacción'...",569000
16258,fotocasa,147,Piso en venta en Berlin,https://www.fotocasa.es/es/comprar/vivienda/co...,https://static.inmofactory.com/images/inmofact...,España,Ciudad 70,NaN,Berlin,28822,...,75,G,NaN,NaN,NaN,NaN,8,0,"{'1': 'Aire acondicionado', '3': 'Calefacción'...",146000
27775,fotocasa,591,"Piso en venta en Calle de la Reina Mercedes, 13",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Tetuán,Cuatro Caminos - Azca,"Calle de la Reina Mercedes, 13",28020,...,140,G,NaN,NaN,Muy bien,50 a 70 años,0,0,"{'2': 'Armarios', '3': 'Calefacción', '9': 'Pa...",720000
24566,fotocasa,677,"Piso en venta en Calle Pío XII, 16",https://www.fotocasa.es/es/comprar/vivienda/co...,https://static.fotocasa.es/images/anuncio/0001...,España,Villalba Estación,NaN,"Calle Pío XII, 16",28400,...,90,G,NaN,NaN,Bien,30 a 50 años,0,0,"{'10': 'Terraza', '32': 'Balcón'}",180000


In [9]:
fotocasa_df.isna().any()

origin               False
page                 False
title                False
link                 False
image_url            False
country              False
district              True
neighborhood          True
street                True
zipCode               True
province             False
buildingType         False
clientAlias           True
latitude             False
longitude            False
isNewConstruction    False
rooms                False
bathrooms            False
parking               True
elevator              True
furnished             True
surface              False
energyCertificate    False
hotWater              True
heating               True
conservationState     True
antiquity             True
floor                False
surfaceLand          False
otherFeatures        False
price                False
dtype: bool

Looks like fotocasa dataset has some NaN values that need to be cleaned out, so let's drop rows with all NaN values:

In [10]:
fotocasa_df = fotocasa_df.dropna(how="all")

In [11]:
fotocasa_df.isna().any()

origin               False
page                 False
title                False
link                 False
image_url            False
country              False
district              True
neighborhood          True
street                True
zipCode               True
province             False
buildingType         False
clientAlias           True
latitude             False
longitude            False
isNewConstruction    False
rooms                False
bathrooms            False
parking               True
elevator              True
furnished             True
surface              False
energyCertificate    False
hotWater              True
heating               True
conservationState     True
antiquity             True
floor                False
surfaceLand          False
otherFeatures        False
price                False
dtype: bool

There are still some NaN values that can be checked later.

In [12]:
fotocasa_df.dtypes

origin               object
page                  int64
title                object
link                 object
image_url            object
country              object
district             object
neighborhood         object
street               object
zipCode              object
province             object
buildingType         object
clientAlias          object
latitude             object
longitude            object
isNewConstruction    object
rooms                object
bathrooms            object
parking              object
elevator             object
furnished            object
surface              object
energyCertificate    object
hotWater             object
heating              object
conservationState    object
antiquity            object
floor                object
surfaceLand          object
otherFeatures        object
price                 int64
dtype: object

Let's compare the two dataset's features

| # | idealista | fotocasa | comment |
| --- | --- | --- | --- |
| 00 | / | origin | transform idealista |
| 01 | / | page | drop |
| 02 | title | title | drop |
| 03 | id | link | transform fotocasa |
| 04 | locationId | / | drop |
| 05 | / | image_url | drop |
| 06 | / | country | drop |
| 07 | area_name | district | **KEEP** |
| 08 | / | neighborhood | drop |
| 09 | / | street | drop |
| 10 | / | zipCode | drop |
| 11 | / | province | drop |
| 12 | propertyType | buildingType | **KEEP** |
| 13 | / | clientAlias | drop |
| 14 | none | latitude | drop |
| 15 | none | longitude | drop |
| 16 | isNewDevelopment | isNewConstruction | **KEEP** |
| 17 | roomNumber | rooms | **KEEP** |
| 18 | bathNumber | bathrooms | **KEEP** |
| 19 | hasParking | parking | **KEEP** |
| 20 | hasLift | elevator | **KEEP** |
| 21 | hasGarden | / | drop |
| 22 | hasSwimmingPool | / | no info for fotocasa |
| 23 | hasTerrace | / | no info for fotocasa |
| 24 | interiorExterior | / | drop |
| 25 | / | furnished | drop |
| 26 | size | surface | **KEEP** |
| 27 | / | energyCertificate | drop |
| 28 | / | hotWater | drop |
| 29 | heatingType | heating | **KEEP** |
| 30 | isGoodCondition | conservationState | transform fotocasa |
| 31 | isNeedsRenovating | / | transform fotocasa |
| 32 | / | antiquity | drop |
| 33 | floor | floor | **KEEP** |
| 34 | / | surfaceLand | drop |
| 35 | featureTags | otherFeatures | *dig deeper into this* |
| 36 | price | price | **KEEP** |


There are some hidden features in the "otherFeatures" column of the fotocasa dataset. Let's check them out.



In [13]:
fotocasa_df['otherFeatures'].unique()

array(["{'2': 'Armarios', '3': 'Calefacción', '10': 'Terraza'}",
       "{'2': 'Armarios', '3': 'Calefacción', '9': 'Parquet', '11': 'Trastero', '12': 'Z. Comunitaria', '109': 'Lavadero'}",
       "{'1': 'Aire acondicionado', '2': 'Armarios', '9': 'Parquet', '10': 'Terraza', '12': 'Z. Comunitaria', '18': 'Suite - con baño', '22': 'Horno', '52': 'Internet', '84': 'Puerta Blindada', '109': 'Lavadero', '131': 'Cocina Equipada'}",
       ...,
       "{'1': 'Aire acondicionado', '2': 'Armarios', '3': 'Calefacción', '11': 'Trastero', '17': 'Piscina', '84': 'Puerta Blindada', '94': 'Piscina comunitaria', '131': 'Cocina Equipada'}",
       "{'6': 'Gres Cerámica', '10': 'Terraza', '84': 'Puerta Blindada', '109': 'Lavadero', '131': 'Cocina Equipada'}",
       "{'2': 'Armarios', '3': 'Calefacción', '10': 'Terraza', '11': 'Trastero', '12': 'Z. Comunitaria', '15': 'Cocina Office', '18': 'Suite - con baño', '22': 'Horno', '84': 'Puerta Blindada', '94': 'Piscina comunitaria', '131': 'Cocina Equipada'

It is necessary to extract all the features and see if there is any that can be of use

In [78]:
new_features = []
new_features_vector = fotocasa_df['otherFeatures'].str.replace(r"{|}|'[0-9][0-9]?[0-9]?': |'",'', regex=True).str.split(', ')
new_features_vector.apply(lambda x: [new_features.append(features) for features in x if features not in new_features and features != ''])
new_features

['Armarios',
 'Calefacción',
 'Terraza',
 'Parquet',
 'Trastero',
 'Z. Comunitaria',
 'Lavadero',
 'Aire acondicionado',
 'Suite - con baño',
 'Horno',
 'Internet',
 'Puerta Blindada',
 'Cocina Equipada',
 'Serv. portería',
 'Jardín Privado',
 'Videoportero',
 'Zona Infantil',
 'Piscina',
 'Balcón',
 'Zona Deportiva',
 'Patio',
 'Gres Cerámica',
 'Electrodomésticos',
 'Lavadora',
 'Microondas',
 'Nevera',
 'Piscina comunitaria',
 'Cocina Office',
 'TV',
 'Domótica',
 'Ascensor interior',
 'Cuarto para el servicio',
 'Energía Solar',
 'Pista de Tenis',
 'Cuarto lavado plancha',
 'Jacuzzi',
 'Alarma',
 'Gimnasio',
 'Baño de huéspedes',
 'Porche cubierto',
 'Bodega',
 'Muebles de diseño',
 'Música ambiental',
 'Sauna',
 'Casa de invitados',
 'Sala de cine']

There is a weird ^ character that is used as a wildcard during scraping. It must be removed.

In [15]:
fotocasa_df[fotocasa_df['otherFeatures'].str.contains("\^")]

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
64,fotocasa,188,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0
67,fotocasa,188,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0
70,fotocasa,188,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0
82,fotocasa,188,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0
84,fotocasa,188,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29162,fotocasa,191,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0
29165,fotocasa,191,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0
29309,fotocasa,165,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0
29311,fotocasa,165,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0


In [16]:
to_drop = fotocasa_df[fotocasa_df['otherFeatures'].str.contains("\^")].index
fotocasa_df = fotocasa_df.drop(to_drop)
del to_drop

There are some features that can be useful, such as "piscina/piscina comunitaria", "patio/jardin privado", "Terraza", which are present in Idealista as well

In [17]:
fotocasa_df['otherFeatures']

0        {'2': 'Armarios', '3': 'Calefacción', '10': 'T...
1        {'2': 'Armarios', '3': 'Calefacción', '9': 'Pa...
2        {'1': 'Aire acondicionado', '2': 'Armarios', '...
3                {'10': 'Terraza', '28': 'Serv. portería'}
4        {'1': 'Aire acondicionado', '2': 'Armarios', '...
                               ...                        
30318    {'1': 'Aire acondicionado', '2': 'Armarios', '...
30319    {'1': 'Aire acondicionado', '10': 'Terraza', '...
30320    {'1': 'Aire acondicionado', '2': 'Armarios', '...
30321    {'1': 'Aire acondicionado', '2': 'Armarios', '...
30322    {'1': 'Aire acondicionado', '2': 'Armarios', '...
Name: otherFeatures, Length: 29937, dtype: object

In [18]:
useful_features = {
    'hasSwimmingPool':['Piscina','Piscina comunitaria'],
    'hasGarden': ['Patio','Jardín Privado'],
    'hasTerrace':['Terraza']
    }

for key,value in useful_features.items():
  fotocasa_df[key] = fotocasa_df['otherFeatures'].str.contains('|'.join(value)).astype(int)

In [19]:
fotocasa_df[useful_features.keys()]

,hasSwimmingPool,hasGarden,hasTerrace
0,0,0,1
1,0,0,0
2,0,0,1
3,0,0,1
4,0,1,1
...,...,...,...
30318,1,1,1
30319,0,0,1
30320,1,1,0
30321,1,1,0


In [20]:
fotocasa_df.columns

Index(['origin', 'page', 'title', 'link', 'image_url', 'country', 'district',
       'neighborhood', 'street', 'zipCode', 'province', 'buildingType',
       'clientAlias', 'latitude', 'longitude', 'isNewConstruction', 'rooms',
       'bathrooms', 'parking', 'elevator', 'furnished', 'surface',
       'energyCertificate', 'hotWater', 'heating', 'conservationState',
       'antiquity', 'floor', 'surfaceLand', 'otherFeatures', 'price',
       'hasSwimmingPool', 'hasGarden', 'hasTerrace'],
      dtype='object')

Idealista dataset also has a similar feature called "featureTags". Let's dig deeper into it:

In [21]:
idealista_df['featureTags'].unique()

array(["['Lujo']", '[]', "['Apartamento']", "['Lujo\\n\\n\\nVilla']",
       "['Villa']", "['Loft']", "['Casa baja']", "['Casa de piedra']",
       "['Buhardilla']", "['Lujo\\n\\n\\nApartamento']",
       "['Lujo\\n\\n\\nBuhardilla']", "['Lujo\\n\\n\\nLoft']"],
      dtype=object)

Apparently there is no new or interesting info into this feature.

Let's keep cleaning the fotocasa_df dataset, starting with the "link".
Fotocasa's links should all look like this, ending with "/d"

In [22]:
fotocasa_df['link'][0]

'https://www.fotocasa.es/es/comprar/vivienda/madrid-capital/calefaccion-terraza-ascensor/164109685/d'

As sanity check, let's see if there is any link that does not respect this rule

In [23]:
fotocasa_df[fotocasa_df['link'].str.endswith('/d')==False]

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price,hasSwimmingPool,hasGarden,hasTerrace


Apparently there is no issue in this colum so it can be transformed into a property id feature

In [24]:
fotocasa_df['id'] = fotocasa_df['link'].apply(lambda x: x.split('/')[-2])
fotocasa_df['id']

0        164109685
1        163938977
2        163368493
3        164116246
4        164075236
           ...    
30318    164015185
30319    163898448
30320    163592870
30321    164091880
30322    163389966
Name: id, Length: 29937, dtype: object

Fotocasa properties' size is separated into surface (representing the actual size of the house) and surfaceLand (the surface of the land on which the house is built on). To have the full size of the property, we need to sum them

In [25]:
fotocasa_df.dtypes

origin               object
page                  int64
title                object
link                 object
image_url            object
country              object
district             object
neighborhood         object
street               object
zipCode              object
province             object
buildingType         object
clientAlias          object
latitude             object
longitude            object
isNewConstruction    object
rooms                object
bathrooms            object
parking              object
elevator             object
furnished            object
surface              object
energyCertificate    object
hotWater             object
heating              object
conservationState    object
antiquity            object
floor                object
surfaceLand          object
otherFeatures        object
price                 int64
hasSwimmingPool       int64
hasGarden             int64
hasTerrace            int64
id                   object
dtype: object

In [26]:
fotocasa_df[fotocasa_df['surfaceLand'] != '0']

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,conservationState,antiquity,floor,surfaceLand,otherFeatures,price,hasSwimmingPool,hasGarden,hasTerrace,id
47,fotocasa,518,"Casa adosada en venta en Calle del Balcón, 9",https://www.fotocasa.es/es/comprar/vivienda/ro...,https://static.fotocasa.es/images/anuncio/2020...,España,NaN,NaN,"Calle del Balcón, 9",28194,...,Muy bien,10 a 20 años,0,20,"{'2': 'Armarios', '10': 'Terraza', '16': 'Pati...",190000,0,1,1,156041669
85,fotocasa,188,Casa o chalet en venta en Río Tajuña,https://www.fotocasa.es/es/comprar/vivienda/es...,https://static.inmofactory.com/images/inmofact...,España,NaN,NaN,Río Tajuña,28595,...,NaN,30 a 50 años,0,2000,{},119995,0,0,0,164566681
111,fotocasa,527,"Casa adosada en venta en Calle Chopo, 20",https://www.fotocasa.es/es/comprar/vivienda/pa...,https://static.fotocasa.es/images/anuncio/2021...,España,NaN,NaN,"Calle Chopo, 20",28189,...,Casi nuevo,10 a 20 años,0,98,"{'2': 'Armarios', '7': 'Jardín Privado', '11':...",390000,0,1,0,159716098
425,fotocasa,712,"Casa o chalet en venta en Calle Granada, 209",https://www.fotocasa.es/es/comprar/vivienda/vi...,https://static.fotocasa.es/images/anuncio/2021...,España,NaN,NaN,"Calle Granada, 209",28630,...,Bien,20 a 30 años,0,1250,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",186000,1,1,1,161297477
441,fotocasa,712,Casa o chalet en venta en Avenida Escondite,https://www.fotocasa.es/es/comprar/vivienda/to...,https://static.fotocasa.es/images/anuncio/2022...,España,Los Peñascales,El Monte,Avenida Escondite,28250,...,A reformar,20 a 30 años,0,800,{},525000,0,0,0,164947061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30120,fotocasa,138,Casa o chalet en venta,https://www.fotocasa.es/es/comprar/vivienda/ce...,https://static.inmofactory.com/images/inmofact...,España,NaN,NaN,NaN,28470,...,Muy bien,30 a 50 años,5,1285,"{'2': 'Armarios', '7': 'Jardín Privado', '10':...",599000,1,1,1,163844263
30132,fotocasa,138,Casa o chalet en venta en Miralrio,https://www.fotocasa.es/es/comprar/vivienda/co...,https://static.inmofactory.com/images/inmofact...,España,NaN,NaN,Miralrio,28380,...,Muy bien,NaN,0,2650,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",325000,1,1,1,158280538
30142,fotocasa,138,Casa o chalet en venta en De Albacete,https://www.fotocasa.es/es/comprar/vivienda/ga...,https://static.inmofactory.com/images/inmofact...,España,El Guijo - Colonia España,NaN,De Albacete,28292,...,Bien,NaN,0,1200,"{'2': 'Armarios', '3': 'Calefacción', '6': 'Gr...",445000,1,1,1,164823857
30158,fotocasa,904,Casa o chalet en venta en Calle Alcalde José M...,https://www.fotocasa.es/es/comprar/vivienda/le...,https://static.fotocasa.es/images/anuncio/2022...,España,Valdepelayo - Montepinos - Arroyo Culebro,NaN,Calle Alcalde José María Durán y Pelayo,28914,...,Muy bien,10 a 20 años,0,250,"{'2': 'Armarios', '3': 'Calefacción', '7': 'Ja...",525000,1,1,1,164454443


In [27]:
print(fotocasa_df['surface'][47])
print(fotocasa_df['surfaceLand'][47])
fotocasa_df['surface'][47] + fotocasa_df['surfaceLand'][47]

211
20


'21120'

In [28]:
fotocasa_df['surface'] = fotocasa_df['surface'].astype(float)
fotocasa_df['surfaceLand'] = fotocasa_df['surfaceLand'].astype(float)
fotocasa_df['surface'][47] + fotocasa_df['surfaceLand'][47]

231.0

In [29]:
fotocasa_df.dtypes

origin                object
page                   int64
title                 object
link                  object
image_url             object
country               object
district              object
neighborhood          object
street                object
zipCode               object
province              object
buildingType          object
clientAlias           object
latitude              object
longitude             object
isNewConstruction     object
rooms                 object
bathrooms             object
parking               object
elevator              object
furnished             object
surface              float64
energyCertificate     object
hotWater              object
heating               object
conservationState     object
antiquity             object
floor                 object
surfaceLand          float64
otherFeatures         object
price                  int64
hasSwimmingPool        int64
hasGarden              int64
hasTerrace             int64
id            

In [30]:
fotocasa_df['size'] = fotocasa_df['surface'] + fotocasa_df['surfaceLand']
fotocasa_df['size']

0         95.0
1        140.0
2        100.0
3        104.0
4        118.0
         ...  
30318    161.0
30319    120.0
30320     87.0
30321    214.0
30322    160.0
Name: size, Length: 29937, dtype: float64

Fotocasa dataset has one feature that informs us on whether the property is in good conditions or needs renovating. Idealista has this information already separated into two True/False features. Let's trasform Fotocasa dataset accordingly:

In [31]:
fotocasa_df['conservationState'].unique()

array(['Muy bien', 'Bien', 'Casi nuevo', nan, 'A reformar', 'Reformado'],
      dtype=object)

"A reformar" means the property needs renovating, while the rest of the values means the house is in good conditions

In [32]:
fotocasa_df['isGoodCondition'] = (fotocasa_df['conservationState']!='A reformar').astype(int)
fotocasa_df['isNeedsRenovating'] = (fotocasa_df['conservationState']=='A reformar').astype(int)

We are going to create a column to identify the real estates of Idealista

In [33]:
fotocasa_df['origin'].head(5)

0    fotocasa
1    fotocasa
2    fotocasa
3    fotocasa
4    fotocasa
Name: origin, dtype: object

In [34]:
idealista_df['origin'] = 'idealista'

Let's compare idealista and fotocasa features again:

In [35]:
idealista_df.columns

Index(['id', 'propertyType', 'title', 'locationId', 'price', 'size',
       'hasParking', 'roomNumber', 'bathNumber', 'hasSwimmingPool',
       'hasTerrace', 'hasGarden', 'hasLift', 'isGoodCondition',
       'isNeedsRenovating', 'isNewDevelopment', 'featureTags', 'heatingType',
       'interiorExterior', 'floor', 'area_name', 'origin'],
      dtype='object')

In [36]:
fotocasa_df.columns

Index(['origin', 'page', 'title', 'link', 'image_url', 'country', 'district',
       'neighborhood', 'street', 'zipCode', 'province', 'buildingType',
       'clientAlias', 'latitude', 'longitude', 'isNewConstruction', 'rooms',
       'bathrooms', 'parking', 'elevator', 'furnished', 'surface',
       'energyCertificate', 'hotWater', 'heating', 'conservationState',
       'antiquity', 'floor', 'surfaceLand', 'otherFeatures', 'price',
       'hasSwimmingPool', 'hasGarden', 'hasTerrace', 'id', 'size',
       'isGoodCondition', 'isNeedsRenovating'],
      dtype='object')

| # | idealista | fotocasa | comment |
| --- | --- | --- | --- |
| 00 | origin | origin | **KEEP** |
| 01 | / | page | drop |
| 02 | title | title | drop |
| 03 | id | id | **KEEP** |
| 04 | locationId | / | drop |
| 05 | / | image_url | drop |
| 06 | / | country | drop |
| 07 | area_name | district | **KEEP** |
| 08 | / | neighborhood | drop |
| 09 | / | street | drop |
| 10 | / | zipCode | drop |
| 11 | / | province | drop |
| 12 | propertyType | buildingType | **KEEP** |
| 13 | / | clientAlias | drop |
| 14 | none | latitude | drop |
| 15 | none | longitude | drop |
| 16 | isNewDevelopment | isNewConstruction | **KEEP** |
| 17 | roomNumber | rooms | **KEEP** |
| 18 | bathNumber | bathrooms | **KEEP** |
| 19 | hasParking | parking | **KEEP** |
| 20 | hasLift | elevator | **KEEP** |
| 21 | hasGarden | hasGarden | **KEEP** |
| 22 | hasSwimmingPool | hasSwimmingPool | **KEEP** |
| 23 | hasTerrace | hasTerrace | **KEEP** |
| 24 | interiorExterior | / | drop |
| 25 | / | furnished | drop |
| 26 | size | size | **KEEP** |
| 27 | / | surface | drop |
| 28 | / | energyCertificate | drop |
| 29 | / | hotWater | drop |
| 30 | heatingType | heating | **KEEP** |
| 31 | / | conservationState | drop |
| 32 | isGoodCondition | isGoodCondition | **KEEP** |
| 33 | isNeedsRenovating | isNeedsRenovating | **KEEP** |
| 34 | / | antiquity | drop |
| 35 | floor | floor | **KEEP** |
| 36 | / | surfaceLand | drop |
| 37 | featureTags | otherFeatures | drop |
| 38 | price | price | **KEEP** |

Having a selected features list now, it is necessary to check them one by one to see if some further data cleaning is needed.

In [37]:
fotocasa_df.isna().any()

origin               False
page                 False
title                False
link                 False
image_url            False
country              False
district              True
neighborhood          True
street                True
zipCode               True
province             False
buildingType         False
clientAlias           True
latitude             False
longitude            False
isNewConstruction    False
rooms                False
bathrooms            False
parking               True
elevator              True
furnished             True
surface              False
energyCertificate    False
hotWater              True
heating               True
conservationState     True
antiquity             True
floor                False
surfaceLand          False
otherFeatures        False
price                False
hasSwimmingPool      False
hasGarden            False
hasTerrace           False
id                   False
size                 False
isGoodCondition      False
i

We can see that there are null values in district, neighborhood, street, zipCode, clientAlias, parking, elevator, furnished, hotWater, heating, conservationState and antiquity.

As many of the features we are going to eliminate, we are only concerned about the ones we are going to keep and have null values, such as district, parking, elevator and heating. Let's take a look at them.

In [38]:
len(fotocasa_df[fotocasa_df['district'].isna()])

2645

In [39]:
fotocasa_df[fotocasa_df['district'].isna()].head()

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surfaceLand,otherFeatures,price,hasSwimmingPool,hasGarden,hasTerrace,id,size,isGoodCondition,isNeedsRenovating
2,fotocasa,940,Piso en venta en Calle Polvoranca,https://www.fotocasa.es/es/comprar/vivienda/hu...,https://static.inmofactory.com/images/inmofact...,España,NaN,NaN,Calle Polvoranca,28970,...,0.0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",200000,0,0,1,163368493,100.0,1,0
17,fotocasa,940,Casa o chalet en venta en Avenida de la Cabrera,https://www.fotocasa.es/es/comprar/vivienda/la...,https://static.inmofactory.com/images/inmofact...,España,NaN,NaN,Avenida de la Cabrera,28751,...,0.0,"{'2': 'Armarios', '6': 'Gres Cerámica', '7': '...",289000,1,1,1,164120295,164.0,1,0
25,fotocasa,940,"Casa o chalet en venta en Calle Real, 9",https://www.fotocasa.es/es/comprar/vivienda/po...,https://static.inmofactory.com/images/inmofact...,España,NaN,NaN,"Calle Real, 9",28813,...,0.0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",380000,0,1,1,162936565,380.0,1,0
45,fotocasa,518,Ático en venta en Calle Madrid,https://www.fotocasa.es/es/comprar/vivienda/hu...,https://static.fotocasa.es/images/anuncio/2021...,España,NaN,NaN,Calle Madrid,28970,...,0.0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",250000,1,1,1,162032193,80.0,1,0
46,fotocasa,518,"Piso en venta en Calle Severo Ochoa, 2",https://www.fotocasa.es/es/comprar/vivienda/hu...,https://static.fotocasa.es/images/anuncio/2022...,España,NaN,NaN,"Calle Severo Ochoa, 2",28970,...,0.0,{'9': 'Parquet'},190000,0,0,0,162571299,86.0,1,0


The postal codes of the districts of the Madrid City Council are: 28013 (Centro), 28045 (Arganzuela), 28007 (Retiro), 28001 (Salamanca), 28002 (Chamartin), 28020 (Tetuán), 28010 (Chamberí), 28029 (Fuencarral - El Pardo), 28008 (Moncloa - Aravaca), 28044 (Latina), 28025 (Carabanchel), 28026 (Usera), 28038 (Puente de Vallecas), 28030 (Moratalaz), 28037 (Ciudad Lineal), 28043 (Hortaleza), 28021 (Villaverde), 28031 (Villa de Vallecas), 28032 (Vicálvaro), 28022 (San Blas - Canillejas) and 28042 (Barajas).

In [40]:
madrid_zipcodes = {'28013':'Centro',
                   '28045':'Arganzuela',
                   '28007':'Retiro',
                   '28001':'Salamanca',
                   '28002':'Chamartin',
                   '28020':'Tetuán',
                   '28010':'Chamberí',
                   '28029':'Fuencarral - El Pardo',
                   '28008':'Moncloa - Aravaca',
                   '28044':'Latina',
                   '28025':'Carabanchel',
                   '28026':'Usera',
                   '28038':'Puente de Vallecas',
                   '28030':'Moratalaz',
                   '28037':'Ciudad Lineal',
                   '28043':'Hortaleza',
                   '28021':'Villaverde',
                   '28031':'Villa de Vallecas',
                   '28032':'Vicálvaro',
                   '28022':'San Blas - Canillejas',
                   '28042':'Barajas'
                   }

In [41]:
list(madrid_zipcodes.keys())

['28013',
 '28045',
 '28007',
 '28001',
 '28002',
 '28020',
 '28010',
 '28029',
 '28008',
 '28044',
 '28025',
 '28026',
 '28038',
 '28030',
 '28037',
 '28043',
 '28021',
 '28031',
 '28032',
 '28022',
 '28042']

In [42]:
len(fotocasa_df[fotocasa_df['zipCode'].isin(list(madrid_zipcodes.keys()))])

7997

We are left with only the Madrid zip codes.

In [161]:
fotocasa_df_madrid = fotocasa_df[fotocasa_df['zipCode'].isin(list(madrid_zipcodes.keys()))]

In [162]:
fotocasa_df_madrid.shape

(7997, 38)

In [163]:
len(fotocasa_df_madrid[fotocasa_df_madrid['district'].isna()])

0

In [164]:
fotocasa_df_madrid['district'].unique()

array(['San Blas', 'Tetuán', 'Arganzuela', 'Puente de Vallecas',
       'Chamartín', 'Fuencarral - El Pardo', 'Villaverde',
       'Villa de Vallecas', 'Latina', 'Carabanchel', 'Retiro',
       'Moncloa - Aravaca', 'Barrio de Salamanca', 'Barajas', 'Centro',
       'Vicálvaro', 'Chamberí', 'Usera', 'Ciudad Lineal', 'Moratalaz',
       'Hortaleza'], dtype=object)

We take a look at parking feature

In [165]:
len(fotocasa_df_madrid[fotocasa_df_madrid['parking'].isna()])

5235

In [166]:
fotocasa_df_madrid['parking'].unique()

array([nan, 'FEATURES.PARKING_PRIVATE', 'FEATURES.PARKING_COMMUNITY'],
      dtype=object)

In [167]:
idealista_df['hasParking'].unique()

array([ 0.,  1., nan])

In [168]:
fotocasa_df_madrid.loc[fotocasa_df_madrid['parking'].isna(),'parking'] = 0
fotocasa_df_madrid.loc[fotocasa_df_madrid['parking'].str.contains('PARKING', na = False),'parking'] = 1
fotocasa_df_madrid['parking'].unique()

array([0, 1], dtype=object)

We take a look at elevator feature

In [169]:
len(fotocasa_df_madrid[fotocasa_df_madrid['elevator'].isna()])

1776

In [170]:
fotocasa_df_madrid['elevator'].unique()

array(['YES', nan], dtype=object)

In [171]:
idealista_df['hasLift'].unique()

array([ 1.,  0., nan])

In [172]:
fotocasa_df_madrid.loc[fotocasa_df_madrid['elevator'].isna(),'elevator'] = 0
fotocasa_df_madrid.loc[fotocasa_df_madrid['elevator'].str.contains('YES', na = False),'elevator'] = 1
fotocasa_df_madrid['elevator'].unique()

array([1, 0], dtype=object)

We take a look heating feature

In [173]:
len(fotocasa_df_madrid[fotocasa_df_madrid['heating'].isna()])

5997

In [174]:
fotocasa_df_madrid['heating'].unique()

array([nan, 'Gas Natural', 'Electricidad', 'Gasóleo', 'Solar', 'Propano'],
      dtype=object)

In [175]:
idealista_df['heatingType'].unique()

array(['Calefacción individual: Gas natural', 'no info',
       'Calefacción individual: Eléctrica', 'Calefacción central',
       'No dispone de calefacción', 'Calefacción central: Gas',
       'Calefacción individual', 'Calefacción central: Gasoil',
       'Calefacción individual: Bomba de frío/calor',
       'Calefacción individual: Gas propano/butano',
       'Calefacción central: Gas natural', 'Calefacción individual: Gas',
       'Calefacción central: Gas propano/butano',
       'Calefacción individual: Gasoil'], dtype=object)

In [176]:
fotocasa_df_madrid.loc[fotocasa_df_madrid['heating'].isna(),'heating'] = 'no info'
fotocasa_df_madrid.loc[fotocasa_df_madrid['heating'] == 'Gas Natural','heating'] = 'Calefacción central: Gas natural'
fotocasa_df_madrid.loc[fotocasa_df_madrid['heating'] == 'Electricidad','heating'] = 'Calefacción individual: Eléctrica'
fotocasa_df_madrid.loc[fotocasa_df_madrid['heating'] == 'Gasóleo','heating'] = 'Calefacción individual: Gasoil'
fotocasa_df_madrid.loc[fotocasa_df_madrid['heating'] == 'Propano','heating'] = 'Calefacción central: Gas propano/butano'

In [177]:
fotocasa_df_madrid['heating'].unique()

array(['no info', 'Calefacción central: Gas natural',
       'Calefacción individual: Eléctrica',
       'Calefacción individual: Gasoil', 'Solar',
       'Calefacción central: Gas propano/butano'], dtype=object)

In [178]:
fotocasa_df_madrid.loc[:,'heating'] = fotocasa_df_madrid['heating'].str.replace(r'^Calefacci[óo]n.*:.','', regex=True)
fotocasa_df_madrid['heating'].unique()

array(['no info', 'Gas natural', 'Eléctrica', 'Gasoil', 'Solar',
       'Gas propano/butano'], dtype=object)

In [194]:
idealista_df.loc[:,'heatingType'] = idealista_df['heatingType'].str.replace(r'^Calefacci[óo]n.*:.','', regex=True)
idealista_df['heatingType'].unique()

array(['Gas natural', 'no info', 'Eléctrica', 'Calefacción central',
       'No dispone de calefacción', 'Gas', 'Calefacción individual',
       'Gasoil', 'Bomba de frío/calor', 'Gas propano/butano'],
      dtype=object)

In [196]:
idealista_df.loc[idealista_df['heatingType'].str.contains('Calefacción central|Calefacción individual', regex = True),'heatingType'] = 'no info'
idealista_df['heatingType'].unique()

array(['Gas natural', 'no info', 'Eléctrica', 'No dispone de calefacción',
       'Gas', 'Gasoil', 'Bomba de frío/calor', 'Gas propano/butano'],
      dtype=object)

The two datasets are quite different. Fotocasa dataset is arguably the most complete an detailed. Still, there are several information we do not really need.

For example, since the project is geographically limited to Madrid area, we do not need several geolocation features that fotocasa provides

In [181]:
fotocasa_df_madrid.columns

Index(['origin', 'page', 'title', 'link', 'image_url', 'country', 'district',
       'neighborhood', 'street', 'zipCode', 'province', 'buildingType',
       'clientAlias', 'latitude', 'longitude', 'isNewConstruction', 'rooms',
       'bathrooms', 'parking', 'elevator', 'furnished', 'surface',
       'energyCertificate', 'hotWater', 'heating', 'conservationState',
       'antiquity', 'floor', 'surfaceLand', 'otherFeatures', 'price',
       'hasSwimmingPool', 'hasGarden', 'hasTerrace', 'id', 'size',
       'isGoodCondition', 'isNeedsRenovating'],
      dtype='object')

In [197]:
fotocasa_df_clean = fotocasa_df_madrid.drop([
    'page',
    'title',
    'link',
    'image_url',
    'country',
    'neighborhood',
    'street',
    'zipCode',
    'province',
    'clientAlias',
    'latitude',
    'longitude',
    'furnished',
    'surface',
    'energyCertificate',
    'hotWater',
    'conservationState',
    'antiquity',
    'surfaceLand',
    'otherFeatures'
    ],axis=1)

In [198]:
idealista_df.columns

Index(['id', 'propertyType', 'title', 'locationId', 'price', 'size',
       'hasParking', 'roomNumber', 'bathNumber', 'hasSwimmingPool',
       'hasTerrace', 'hasGarden', 'hasLift', 'isGoodCondition',
       'isNeedsRenovating', 'isNewDevelopment', 'featureTags', 'heatingType',
       'interiorExterior', 'floor', 'area_name', 'origin'],
      dtype='object')

In [199]:
idealista_df_clean = idealista_df.drop([
    'title',
    'locationId',
    'interiorExterior',
    'featureTags'
],axis=1)

In [200]:
len(fotocasa_df_clean.columns)

18

In [201]:
len(idealista_df_clean.columns)

18

In [202]:
idealista_df_clean.head()

,id,propertyType,price,size,hasParking,roomNumber,bathNumber,hasSwimmingPool,hasTerrace,hasGarden,hasLift,isGoodCondition,isNeedsRenovating,isNewDevelopment,heatingType,floor,area_name,origin
0,98255270.0,piso,995000.0,179.0,0.0,4.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,Gas natural,planta 1ª,Tetuán,idealista
1,94343260.0,piso,99000.0,56.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,no info,no info,Puente de Vallecas,idealista
2,98119783.0,piso,425000.0,31.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,no info,planta 5ª,Centro,idealista
3,97132927.0,piso,136000.0,55.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Eléctrica,planta 3ª,Puente de Vallecas,idealista
4,97611862.0,piso,574300.0,94.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,no info,planta 4ª,Barrio de Salamanca,idealista


Now both datasets have the same features. It is necessary to make sure all column names match before merging them.

In [203]:
idealista_df_clean['district'] = idealista_df_clean['area_name']
idealista_df_clean = idealista_df_clean.drop(['area_name'],axis=1)

column_mapper = {
    'buildingType': 'propertyType',
    'isNewConstruction': 'isNewDevelopment',
    'rooms': 'roomNumber',
    'bathrooms': 'bathNumber',
    'parking': 'hasParking',
    'elevator': 'hasLift',
    'heating': 'heatingType'
}

for key,value in column_mapper.items():
  fotocasa_df_clean[value] = fotocasa_df_clean[key]

fotocasa_df_clean = fotocasa_df_clean.drop(column_mapper.keys(),axis=1)

In [337]:
df = pd.concat([idealista_df_clean,fotocasa_df_clean])

In [338]:
df.columns

Index(['id', 'propertyType', 'price', 'size', 'hasParking', 'roomNumber',
       'bathNumber', 'hasSwimmingPool', 'hasTerrace', 'hasGarden', 'hasLift',
       'isGoodCondition', 'isNeedsRenovating', 'isNewDevelopment',
       'heatingType', 'floor', 'origin', 'district'],
      dtype='object')

In [339]:
df.sample(20)

,id,propertyType,price,size,hasParking,roomNumber,bathNumber,hasSwimmingPool,hasTerrace,hasGarden,hasLift,isGoodCondition,isNeedsRenovating,isNewDevelopment,heatingType,floor,origin,district
9941,98510475.0,piso,137000.0,20.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Eléctrica,bajo,idealista,Centro
76,163707826,Flat,114900.0,50.0,0,2,1,0.0,0.0,0.0,0,1.0,0.0,False,Eléctrica,3,fotocasa,Puente de Vallecas
16527,163389966,Flat,720000.0,160.0,1,4,3,1.0,0.0,0.0,1,1.0,0.0,False,no info,0,fotocasa,Retiro
11322,164065380,Flat,448500.0,93.0,0,2,3,0.0,0.0,1.0,1,1.0,0.0,False,no info,0,fotocasa,Centro
10927,98370273.0,piso,145000.0,40.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,no info,bajo,idealista,Ciudad Lineal
3479,97478596.0,piso,2500000.0,463.0,1.0,4.0,5.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,Gas natural,planta 1ª,idealista,Chamartín
10954,98349446.0,piso,250000.0,91.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,no info,bajo,idealista,Tetuán
14777,88544986.0,piso,160000.0,52.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,no info,no info,idealista,Tetuán
3249,163990374,Flat,339000.0,120.0,1,4,2,0.0,0.0,1.0,1,1.0,0.0,False,no info,0,fotocasa,Carabanchel
23735,164764395,Flat,220000.0,40.0,0,1,1,0.0,0.0,0.0,1,1.0,0.0,False,Gas natural,5,fotocasa,Tetuán


We must now maintain coherence between the values of each feature

In [340]:
df = df.reset_index(drop=True)

In [341]:
df.head()

,id,propertyType,price,size,hasParking,roomNumber,bathNumber,hasSwimmingPool,hasTerrace,hasGarden,hasLift,isGoodCondition,isNeedsRenovating,isNewDevelopment,heatingType,floor,origin,district
0,98255270.0,piso,995000.0,179.0,0.0,4.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,Gas natural,planta 1ª,idealista,Tetuán
1,94343260.0,piso,99000.0,56.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,no info,no info,idealista,Puente de Vallecas
2,98119783.0,piso,425000.0,31.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,no info,planta 5ª,idealista,Centro
3,97132927.0,piso,136000.0,55.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Eléctrica,planta 3ª,idealista,Puente de Vallecas
4,97611862.0,piso,574300.0,94.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,no info,planta 4ª,idealista,Barrio de Salamanca


In [342]:
df.isna().any()

id                    True
propertyType          True
price                 True
size                  True
hasParking            True
roomNumber            True
bathNumber            True
hasSwimmingPool       True
hasTerrace            True
hasGarden             True
hasLift               True
isGoodCondition       True
isNeedsRenovating     True
isNewDevelopment      True
heatingType          False
floor                 True
origin                True
district              True
dtype: bool

In [343]:
df[df['id'].isna()]

,id,propertyType,price,size,hasParking,roomNumber,bathNumber,hasSwimmingPool,hasTerrace,hasGarden,hasLift,isGoodCondition,isNeedsRenovating,isNewDevelopment,heatingType,floor,origin,district
16390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no info,NaN,NaN,NaN


In [344]:
df = df.drop([16390],axis = 0)

In [345]:
df = df.reset_index(drop=True)
df.isna().any()

id                   False
propertyType         False
price                False
size                 False
hasParking           False
roomNumber           False
bathNumber           False
hasSwimmingPool      False
hasTerrace           False
hasGarden            False
hasLift              False
isGoodCondition      False
isNeedsRenovating    False
isNewDevelopment     False
heatingType          False
floor                False
origin               False
district             False
dtype: bool

In [346]:
df.sample(10)

,id,propertyType,price,size,hasParking,roomNumber,bathNumber,hasSwimmingPool,hasTerrace,hasGarden,hasLift,isGoodCondition,isNeedsRenovating,isNewDevelopment,heatingType,floor,origin,district
7428,96981913.0,piso,154000.0,89.0,0.0,3.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,Gas natural,planta 1ª,idealista,Puente de Vallecas
18374,164020487,Flat,315000.0,97.0,0,3,2,0.0,1.0,1.0,0,1.0,0.0,False,no info,0,fotocasa,Usera
12552,98508554.0,piso,220000.0,50.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Gas natural,bajo,idealista,Tetuán
6631,96944529.0,piso,330000.0,89.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,Gas natural,planta 3ª,idealista,Ciudad Lineal
1630,97346104.0,piso,250000.0,76.0,1.0,3.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,Gas natural,entreplanta,idealista,Latina
1438,97554905.0,piso,1625000.0,212.0,0.0,2.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,no info,planta 6ª,idealista,Barrio de Salamanca
1765,98417607.0,piso,875000.0,159.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,Gas natural,planta 1ª,idealista,Centro
2198,81854746.0,chalet,3800000.0,674.0,0.0,5.0,6.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,no info,does not apply,idealista,Moncloa
9960,91351612.0,piso,587000.0,142.0,1.0,3.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,Gas natural,planta 3ª,idealista,Fuencarral
16479,164078866,Flat,340000.0,143.0,1,3,2,0.0,0.0,0.0,1,1.0,0.0,False,no info,0,fotocasa,Villaverde


In [347]:
df['propertyType'].unique()

array(['piso', 'chalet', 'casa rural', 'Flat'], dtype=object)

In [348]:
# propertyType transformations
df.loc[df['propertyType'].str.contains('Flat|piso', regex = True),'propertyType'] = 'flat'
df.loc[df['propertyType'] == 'casa rural','propertyType'] = 'rural house'
df['propertyType'].unique()

array(['flat', 'chalet', 'rural house'], dtype=object)

In [349]:
df['floor'].unique()

array(['planta 1ª', 'no info', 'planta 5ª', 'planta 3ª', 'planta 4ª',
       'does not apply', 'bajo', 'planta 6ª', 'planta 2ª', 'planta 14ª',
       'planta 10ª', 'planta 9ª', 'planta 7ª', 'planta 8ª', 'entreplanta',
       'semi-sótano', 'planta -1', 'planta 15ª', 'planta 20ª', 'sótano',
       'planta 13ª', 'planta 12ª', 'planta 11ª', 'planta 16ª',
       'planta 19ª', 'planta 18ª', 'planta 40ª', 'planta 17ª',
       'planta 26ª', 'planta 21ª', '0', '7', '10', '8', '3', '11', '12',
       '6', '9', '15', '14', '2', '13', '5', '16', '4', '17', '18', '19',
       '20', '21'], dtype=object)

In [350]:
# floor transformations
df.loc[:,'floor'] = df['floor'].str.replace(r'planta.|ª','', regex=True)
df['floor'].unique()

array(['1', 'no info', '5', '3', '4', 'does not apply', 'bajo', '6', '2',
       '14', '10', '9', '7', '8', 'entreplanta', 'semi-sótano', '-1',
       '15', '20', 'sótano', '13', '12', '11', '16', '19', '18', '40',
       '17', '26', '21', '0'], dtype=object)

In [351]:
df[df['floor'] == 'bajo']['propertyType'].unique()

array(['flat'], dtype=object)

In [352]:
df.loc[:,'floor'] = df['floor'].str.replace(r'bajo','0', regex=True)

In [353]:
df[df['floor'] == 'does not apply']['propertyType'].unique()

array(['chalet', 'rural house'], dtype=object)

In [354]:
# isNewDevelopment transformations
df['isNewDevelopment'].unique()

array([0.0, 1.0, 'False'], dtype=object)

In [355]:
df.loc[df['isNewDevelopment'] == 'False','isNewDevelopment'] = 0.0

In [356]:
df['isNewDevelopment'].unique()

array([0.0, 1.0], dtype=object)

In [364]:
# district transformations
df['district'].unique()

array(['Tetuán', 'Puente de Vallecas', 'Centro', 'Barrio de Salamanca',
       'Chamberí', 'San Blas', 'Villaverde', 'Vicálvaro', 'Carabanchel',
       'Fuencarral', 'Retiro', 'Chamartín', 'Arganzuela', 'Moncloa',
       'Villa de Vallecas', 'Hortaleza', 'Latina', 'Ciudad Lineal',
       'Usera', 'Barajas', 'Moratalaz', 'Fuencarral - El Pardo',
       'Moncloa - Aravaca'], dtype=object)

In [369]:
df.loc[df['district'] == 'Moncloa - Aravaca','district'] = 'Moncloa'
df.loc[df['district'] == 'Fuencarral - El Pardo','district'] = 'Fuencarral'
df.loc[df['district'] == 'Barrio de Salamanca','district'] = 'Salamanca'

In [370]:
df['hasParking'] = df['hasParking'].astype('int64')
df['roomNumber'] = df['roomNumber'].astype('int64')
df['bathNumber'] = df['bathNumber'].astype('int64')
df['hasSwimmingPool'] = df['hasSwimmingPool'].astype('int64')
df['hasTerrace'] = df['hasTerrace'].astype('int64')
df['hasGarden'] = df['hasGarden'].astype('int64')
df['hasLift'] = df['hasLift'].astype('int64')
df['isGoodCondition'] = df['isGoodCondition'].astype('int64')
df['isNeedsRenovating'] = df['isNeedsRenovating'].astype('int64')
df['isNewDevelopment'] = df['isNewDevelopment'].astype('int64')

In [374]:
price_m2_per_area = df[['district','price_m2']].groupby(['district']).median().reset_index()
price_m2_per_area = {key:value for key,value in price_m2_per_area.values}

price_m2_per_area

{'Arganzuela': 3860.29,
 'Barajas': 3437.5,
 'Carabanchel': 2500.0,
 'Centro': 5331.715,
 'Chamartín': 6086.96,
 'Chamberí': 6027.355,
 'Ciudad Lineal': 3434.8450000000003,
 'Fuencarral': 4315.045,
 'Hortaleza': 4166.67,
 'Latina': 2214.29,
 'Moncloa': 4418.6,
 'Moratalaz': 2666.67,
 'Puente de Vallecas': 2481.48,
 'Retiro': 4657.995,
 'Salamanca': 7474.595,
 'San Blas': 2461.54,
 'Tetuán': 4591.48,
 'Usera': 2688.1,
 'Vicálvaro': 2912.4700000000003,
 'Villa de Vallecas': 2617.51,
 'Villaverde': 2150.95}

In [375]:
df['price_m2'] = (df['price'] / df['size']).round(2)

# Idealista often suffer from fraudolent announces so it's better to aggregate 
# the price_m2_per_area using the median as it is more robust than the mean
price_m2_per_area = df[['district','price_m2']].groupby(['district']).median().reset_index()
price_m2_per_area = {key:value for key,value in price_m2_per_area.values}

df['district_price'] = df['district'].map(price_m2_per_area).astype(float).round(2)

In [376]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24387 entries, 0 to 24386
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 24387 non-null  object 
 1   propertyType       24387 non-null  object 
 2   price              24387 non-null  float64
 3   size               24387 non-null  float64
 4   hasParking         24387 non-null  int64  
 5   roomNumber         24387 non-null  int64  
 6   bathNumber         24387 non-null  int64  
 7   hasSwimmingPool    24387 non-null  int64  
 8   hasTerrace         24387 non-null  int64  
 9   hasGarden          24387 non-null  int64  
 10  hasLift            24387 non-null  int64  
 11  isGoodCondition    24387 non-null  int64  
 12  isNeedsRenovating  24387 non-null  int64  
 13  isNewDevelopment   24387 non-null  int64  
 14  heatingType        24387 non-null  object 
 15  floor              24387 non-null  object 
 16  origin             243

In [378]:
df

,id,propertyType,price,size,hasParking,roomNumber,bathNumber,hasSwimmingPool,hasTerrace,hasGarden,hasLift,isGoodCondition,isNeedsRenovating,isNewDevelopment,heatingType,floor,origin,district,price_m2,district_price
0,98255270.0,flat,995000.0,179.0,0,4,5,0,0,0,1,1,0,0,Gas natural,1,idealista,Tetuán,5558.66,4591.48
1,94343260.0,flat,99000.0,56.0,0,3,1,0,0,0,0,1,0,0,no info,no info,idealista,Puente de Vallecas,1767.86,2481.48
2,98119783.0,flat,425000.0,31.0,0,2,1,0,0,0,1,1,0,0,no info,5,idealista,Centro,13709.68,5331.72
3,97132927.0,flat,136000.0,55.0,0,2,1,0,0,0,0,1,0,0,Eléctrica,3,idealista,Puente de Vallecas,2472.73,2481.48
4,97611862.0,flat,574300.0,94.0,0,4,1,0,0,0,1,1,0,0,no info,4,idealista,Salamanca,6109.57,7474.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24382,164078866,flat,340000.0,143.0,1,3,2,0,0,0,1,1,0,0,no info,0,fotocasa,Villaverde,2377.62,2150.95
24383,160436162,flat,239000.0,89.0,1,3,2,0,0,0,1,1,0,0,no info,0,fotocasa,Villa de Vallecas,2685.39,2617.51
24384,164083993,flat,299999.0,105.0,0,3,2,0,1,0,1,1,0,0,no info,0,fotocasa,Puente de Vallecas,2857.13,2481.48
24385,164015185,flat,750000.0,161.0,0,3,3,1,1,1,1,1,0,0,no info,0,fotocasa,Fuencarral,4658.39,4315.04
